speed

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
%matplotlib inline
%config InlineBackend.figure_format='retina'
import cmocean
import cartopy.crs as ccrs
import matplotlib.path as mpath
import nc_time_axis
import pandas as pd
import matplotlib as mpl
import momlevel as ml
import gcsfs
import zarr
from matplotlib.ticker import FormatStrFormatter

In [13]:
mask_CM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/CM4_mask_antarctica.nc')
mask_ESM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ESM4_mask_antarctica.nc')
mask_ERA5= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ERA5_mask_antarctica.nc')

In [3]:
##importing Eastward Near-Surface Wind data from google cloud. Amon
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
	
# step two … # replace with the id you need and other info to ween down datasets 
df_sfw = df.query("source_id=='GFDL-CM4' & experiment_id == 'historical' & variable_id == 'sfcWind'& table_id == 'Amon'")



In [4]:
df_u_historical

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
6284,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Amon,sfcWind,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701


In [4]:
# step three - loading in the data and storing it
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_sfw.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_sfw = xr.open_zarr(mapper, consolidated=True)


In [17]:
ds_sfw

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 1980)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    height     float64 ...
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Data variables:
    sfcWind    (time, lat, lon) float32 dask.array<chunksize=(327, 180, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  36500.0
    comment:                <null ref>
    ...                     ...
    tracking_id:            hdl:21.14100/e319729b-f0a9-4b0d-9fe1-101499666a93...
    variable_id:            sfcWind
    variant_info:           N/A
    variant_label:          r1i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/e319729b-f0a9-4b0d-9fe1-101499666a93...
    version_id:             v20180701

In [14]:
overall_cm4_speed=ds_sfw.sel(time=slice('1995-01-01','2014-12-01')).mean(dim='time').sel(lat=slice(-89.5,-55))

In [10]:
sfw_seasonal=ds_sfw.sfcWind.sel(time=slice('1995-01-01','2014-12-01')).groupby('time.season').mean(dim='time').sel(lat=slice(-89.5,-55))
sfw_seasonal

<xarray.DataArray 'sfcWind' (season: 4, lat: 35, lon: 288)>
dask.array<getitem, shape=(4, 35, 288), dtype=float32, chunksize=(1, 35, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 ... -58.5 -57.5 -56.5 -55.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Near-Surface Wind Speed
    original_name:  sfcWind
    standard_name:  wind_speed
    units:          m s-1

In [11]:
sfw_cm4_summer=sfw_seasonal.sel(season='DJF')
sfw_cm4_winter=sfw_seasonal.sel(season='JJA')
sfw_cm4_spring=sfw_seasonal.sel(season='SON')
sfw_cm4_fall=sfw_seasonal.sel(season='MAM')

In [12]:
sfw_cm4_summer

<xarray.DataArray 'sfcWind' (lat: 35, lon: 288)>
dask.array<getitem, shape=(35, 288), dtype=float32, chunksize=(35, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 ... -58.5 -57.5 -56.5 -55.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
    season   <U3 'DJF'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Near-Surface Wind Speed
    original_name:  sfcWind
    standard_name:  wind_speed
    units:          m s-1

In [15]:
plateau_wind_total_CM4=(overall_cm4_speed*(mask_CM4.plateau_CM4)).mean(dim='lat').mean(dim='lon')
ross_wind_total_CM4=(overall_cm4_speed*(mask_CM4.ross_CM4)).mean(dim='lat').mean(dim='lon')
lowwest_wind_total_CM4=(overall_cm4_speed*(mask_CM4.lowwest_CM4)).mean(dim='lat').mean(dim='lon')
loweast_wind_total_CM4=(overall_cm4_speed*(mask_CM4.loweast_CM4)).mean(dim='lat').mean(dim='lon')
filchner_wind_total_CM4=(overall_cm4_speed*(mask_CM4.filchner_CM4)).mean(dim='lat').mean(dim='lon')
peninsula_wind_total_CM4=(overall_cm4_speed*(mask_CM4.peninsula_CM4)).mean(dim='lat').mean(dim='lon')


In [16]:
plat_value_CM4_total=print(plateau_wind_total_CM4.values)
ross_value_CM4_total= print(ross_wind_total_CM4.values)
west_value_CM4_total=print(lowwest_wind_total_CM4.values)
east_value_CM4_total=print(loweast_wind_total_CM4.values)
filch_value_CM4_total=print(filchner_wind_total_CM4.values)
pen_value_CM4_total=print(peninsula_wind_total_CM4.values)

<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (bnds: 2)
Coordinates:
  * bnds     (bnds) float64 1.0 2.0
    height   float64 ...
Data variables:
    sfcWind  float32 dask.array<chunksize=(), meta=np.ndarray>>
<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (bnds: 2)
Coordinates:
  * bnds     (bnds) float64 1.0 2.0
    height   float64 ...
Data variables:
    sfcWind  float32 dask.array<chunksize=(), meta=np.ndarray>>
<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (bnds: 2)
Coordinates:
  * bnds     (bnds) float64 1.0 2.0
    height   float64 ...
Data variables:
    sfcWind  float32 dask.array<chunksize=(), meta=np.ndarray>>
<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (bnds: 2)
Coordinates:
  * bnds     (bnds) float64 1.0 2.0
    height   float64 ...
Data variables:
    sfcWind  float32 dask.array<chunksize=(), meta=np.ndarray>>
<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (bnds: 2)
Coordinates:
  * bnds   